In [10]:
import torch
import torchvision
import torchvision.transforms as tr 
from torch.utils.data import DataLoader, Dataset 

### pytorch 내장 데이터 불러오기

In [11]:
transf = tr.Compose([tr.ToTensor(),
                     tr.Normalize((0.5, 0.5, 0.5),(0.5,0.5,0.5))]) # torch 내장 데이터(pil이미지를 받는것이므로)니까 그냥 내장 ToTensor 이용해도 됨

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transf)
trainloader = DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transf)
testloader = DataLoader(testset, batch_size=8, shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### Build a model

In [12]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    # 1. 연산셋팅
    def __init__(self): 
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) # output이 10개의 종류로 나옴

    # 2. 연산순서정의        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5) # fully connected 에 넣기위해 1자로 피는 작업임
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net() # class에 대한 instance 생성 model = Model() 딱 이거랑 같은거임

In [13]:
print(net)
# 32  -> 28 -> 14 -> 10 -> 5

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### Implement the model with training set

In [14]:
import torch.optim as optim

# 1. 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 분류문제에서 흔히쓰이는 loss function
# 2. optimizer 정의
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # net.parameters() : 위의 instance에 쓰이는 모든 파라미터들 의미

In [18]:
epoch_num = 3 # 'epoch_num' : 전체를 몇 회독 할 것이냐는 의미
for epoch in range(epoch_num) :
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0): # trainloader : 이 안에 이미지 8장씩 있음
        
        images, labels = data
        
        # paramerter 0으로 초기설정
        optimizer.zero_grad() 
        
        outputs = net(images) # 이 images가 forward에서 x가 되는셈!!
        loss = loss_function(outputs, labels) # 배치 사이즈 8장에 대한 loss값(스칼라)
        loss.backward() # loss기준으로 backward
        optimizer.step() # opmizer 계산한다
        
        # print log
        running_loss += loss.item() # 이렇게 하면 스칼라값 표현가능
        if i % 2000 == 1999:
            # print every 2000 mini-batches
            print('[%d, %5d] loss : %.3f' %
                  (epoch +1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss : 1.408
[1,  4000] loss : 1.382
[1,  6000] loss : 1.332
[2,  2000] loss : 1.266
[2,  4000] loss : 1.224
[2,  6000] loss : 1.216
[3,  2000] loss : 1.158
[3,  4000] loss : 1.141
[3,  6000] loss : 1.124
Finished Training


- 위의 SGD에서는 lr이 고정되어있음
- 이때 scheduler라는걸 사용할 수 있음
```
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1, momentum = 0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1) # batch 30번마다 lr *= 0.1 하겠단거임

for epoch in range(epoch_num)
    pass
    for data in trainloader :
        pass
    scheduler.step() # 이렇게 epoch에 대한 for문에 넣어야함. for문 위치 주의
```

### Save the trained model
- path = ''
- torch.save(net.state_dict(), path)

### Load the pre-trained model
- net = Net()
- net.load_state_dict(torch.load(path))

### Predict test data
- output은 미니배치의 결과가 산출되는 것이므로 for문을 통해서 test 전체의 예측값을 구해야 함

In [19]:
correct = 0
total = 0
with torch.no_grad() : # test data 는 gradient update를 안하므로
    for data in testloader :
        images, labels = data
        outputs = net(images) # outputs : 10개의 값이 담겨있는 벡터
        _, predicted = torch.max(outputs.data, 1) # 그 중 가장 큰 값을 고른다
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuaracy of the network on the 10,000 test images : %d %%' % (
    100 * correct / total))

Accuaracy of the network on the 10,000 test images : 58 %
